In [ ]:
!mkdir -p /content/drive


In [ ]:
from google.colab import drive
from keras.preprocessing.image import ImageDataGenerator
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

# List contents of the root directory in Google Drive
root_contents = os.listdir('/content/drive/MyDrive')
print("Contents of the root directory:")
print(root_contents)


Contents of the root directory:
['Classroom', '211020451_assignment.zip', '211020451_ShristiTiwari.zip', 'Shristi_211020451 (1).pdf', '211020451ass.zip', 'akriti.211010209.zip', 'Shristi_211020451.pdf', '211020451_ass3.pdf', '16431025075258118902503112517639.jpg', '16431027455436902428119220904452.jpg', '211020451_ass4.pdf', '211020451_Shristi_ass3 (1).zip', '211020451_Shristi_ass3.zip', '211020451_ass3 (1).zip', '211020451_ass3.zip', '211020451-ass4.pdf', '211020451_shristi_dldass1.pdf', '211020451_report.pdf', 'Pointers.pdf', '211020451_ass5(C).pdf', 'File-Programs-25-02-2022-updated.docx', 'ques.jpg', 'Interview ', '211020451_ass6.pdf', 'Ass2DLD.pdf', 'Document-Shristi Tiwari.pdf', 'Document.gdoc', 'Document-Shristi Tiwari.gdoc', 'ques1DA.pdf', 'ques2DA.pdf', 'ques3DA.pdf', 'ques5DA.pdf', 'ques6DA.pdf', 'ques4DA.pdf', 'Untitled document (9).gdoc', 'Untitled document (8).gdoc', 'Untitled document (7).gdoc', 'Untitled document (6).gdoc', 'Screenshot_2022-05-17-14-45-06-44_6012fa4d4dde

In [ ]:
path_to_imagery = '/content/drive/MyDrive/Imagery'
train_data_dir = '/content/drive/MyDrive/Imagery/train'
val_data_dir = '/content/drive/MyDrive/Imagery/val'
test_data_dir = '/content/drive/MyDrive/Imagery/test'

In [ ]:
print("Training Data Directory:", train_data_dir)
print("Validation Data Directory:", val_data_dir)
print("Test Data Directory:", test_data_dir)


Training Data Directory: /content/drive/MyDrive/Imagery/train
Validation Data Directory: /content/drive/MyDrive/Imagery/val
Test Data Directory: /content/drive/MyDrive/Imagery/test


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

img_width, img_height = 150, 150
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)


Found 475 images belonging to 2 classes.
Found 843 images belonging to 2 classes.
Found 831 images belonging to 2 classes.


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# Define image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Data augmentation and normalization for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Normalization for validation and test data (no augmentation)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Define the paths to your data folders
train_data_dir = '/content/drive/MyDrive/Imagery/train'
validation_data_dir = '/content/drive/MyDrive/Imagery/val'
test_data_dir = '/content/drive/MyDrive/Imagery/test'

# Load and prepare training data with target segmentation masks
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',  # Specify 'input' for segmentation masks
    color_mode='rgb',  # Assuming RGB images
    shuffle=True  # Shuffle the data
)

# Load and prepare validation data with target segmentation masks
validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',  # Specify 'input' for segmentation masks
    color_mode='rgb',  # Assuming RGB images
    shuffle=False  # Do not shuffle validation data
)

# Load and prepare test data without target data (only input images)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,  # No class mode for test data
    color_mode='rgb',  # Assuming RGB images
    shuffle=False  # Do not shuffle test data
)


Found 475 images belonging to 2 classes.
Found 843 images belonging to 2 classes.
Found 831 images belonging to 2 classes.


In [ ]:
# Check the structure of the yielded batches from the generator
for batch in train_generator:
    print('Batch type:', type(batch))
    print('Batch length:', len(batch))
    print('Batch data shape:', batch[0].shape)
    print('Batch labels shape:', batch[1].shape)
    break  # Only print the structure of the first batch



Batch type: <class 'tuple'>
Batch length: 2
Batch data shape: (32, 150, 150, 3)
Batch labels shape: (32,)


In [ ]:
for data_batch, labels_batch in train_generator:
    print('Data batch shape:', data_batch.shape)
    print('Labels batch shape:', labels_batch.shape)
    break


Data batch shape: (32, 150, 150, 3)
Labels batch shape: (32,)


In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator

# Define image dimensions, batch size, epochs, and steps per epoch
num_classes = 2  # For binary classification with 2 classes
img_width, img_height = 150, 150
batch_size = 32
epochs = 10

# Data augmentation and normalization for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Load and prepare training data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,  # Directory containing training images
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'  # For binary classification
)

validation_generator = train_datagen.flow_from_directory(
    validation_data_dir,  # Directory containing validation images
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'  # For binary classification
)

# Define the model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))  # Binary classification with 2 classes, using softmax

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


Found 475 images belonging to 2 classes.
Found 843 images belonging to 2 classes.


In [ ]:
# Assuming you have already defined and compiled your model and data generators
from keras.callbacks import ModelCheckpoint

# Define image dimensions, batch size, epochs, and steps per epoch
num_classes = 2  # Assuming you have 2 classes
img_width, img_height = 150, 150
batch_size = 32
epochs = 10

steps_per_epoch = len(train_generator)
validation_steps = len(validation_generator)
checkpoint_path = '/content/drive/MyDrive/Imagery/checkpoints/model_checkpoint.h5'
checkpoint_callback = ModelCheckpoint(
    checkpoint_path,
    monitor='val_accuracy',  # You can choose the metric to monitor
    save_best_only=True,
    mode='max',
    verbose=1
)

# Fit the model using the generators and the checkpoint callback
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=[checkpoint_callback]  # Include the checkpoint callback in the callbacks list
)

Epoch 1/10
15/15 [==============================] - ETA: 0s - loss: 0.1896 - accuracy: 0.9179
Epoch 1: val_accuracy improved from -inf to 1.00000, saving model to /content/drive/MyDrive/Imagery/checkpoints/model_checkpoint.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


15/15 [==============================] - 1038s 73s/step - loss: 0.1896 - accuracy: 0.9179 - val_loss: 0.0270 - val_accuracy: 1.0000
Epoch 2/10
15/15 [==============================] - ETA: 0s - loss: 0.0017 - accuracy: 1.0000
Epoch 2: val_accuracy did not improve from 1.00000
15/15 [==============================] - 236s 16s/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 2.7888e-05 - val_accuracy: 1.0000
Epoch 3/10
15/15 [==============================] - ETA: 0s - loss: 1.7620e-06 - accuracy: 1.0000
Epoch 3: val_accuracy did not improve from 1.00000
15/15 [==============================] - 222s 15s/step - loss: 1.7620e-06 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/10
15/15 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4: val_accuracy did not improve from 1.00000
15/15 [==============================] - 210s 15s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/10
1

In [ ]:
from keras.callbacks import ModelCheckpoint

# Define image dimensions, batch size, epochs, and steps per epoch
num_classes = 2  # Assuming you have 2 classes
img_width, img_height = 150, 150
batch_size = 32
epochs = 10

# Assuming you have already defined and compiled your model and data generators
steps_per_epoch = len(train_generator)
validation_steps = len(validation_generator)

# Define the path to save model checkpoints in .h5 format
checkpoint_path_h5 = '/content/drive/MyDrive/Imagery/checkpoints/model_checkpoint.h5'
checkpoint_callback_h5 = ModelCheckpoint(
    checkpoint_path_h5,
    monitor='val_accuracy',  # You can choose the metric to monitor
    save_best_only=True,
    mode='max',
    verbose=1
)

# Define the path to save model checkpoints in .ckpt format (TensorFlow format)
checkpoint_path_ckpt = '/content/drive/MyDrive/Imagery/checkpoints/model_checkpoint.ckpt'
checkpoint_callback_ckpt = ModelCheckpoint(
    checkpoint_path_ckpt,
    monitor='val_accuracy',  # You can choose the metric to monitor
    save_best_only=True,
    mode='max',
    save_weights_only=True,  # Save only the weights in TensorFlow format
    verbose=1
)

# Define other possible options for saving checkpoints, e.g., specifying save_freq
checkpoint_path_custom = '/content/drive/MyDrive/Imagery/checkpoints/model_checkpoint_{epoch:02d}.h5'
checkpoint_callback_custom = ModelCheckpoint(
    checkpoint_path_custom,
    monitor='val_accuracy',  # You can choose the metric to monitor
    save_best_only=True,
    mode='max',
    save_freq='epoch',  # Save every epoch
    verbose=1
)

# Fit the model using the generators and the checkpoint callbacks
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=[checkpoint_callback_h5, checkpoint_callback_ckpt, checkpoint_callback_custom]
    # Include the checkpoint callbacks in the callbacks list
)


In [ ]:
from keras.preprocessing import image
import numpy as np
num_classes = 2  # For binary classification with 2 classes
img_width, img_height = 150, 150
# Define the path to the image you want to test
image_path = '/content/7143.jpg'

# Load the image and preprocess it
img = image.load_img(image_path, target_size=(img_width, img_height))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array /= 255.0  # Normalize the pixel values

# Make predictions using your trained model
predictions = model.predict(img_array)

# Get the predicted class label
predicted_class = np.argmax(predictions)

# Print the predicted class
if predicted_class == 0:
    print("The image belongs to Class 0 (Minor Damage).")
else:
    print("The image belongs to Class 1 (Major Damage).")
